In [0]:
select * from gap_catalog.ads_owner.EVENT_STATUS
where est_key||'.'||est_valid_from = '1.2025-08-14 00:00:00'
order by est_key;

select * from
gap_catalog.ads_etl_owner.XC_RDS_EVENT_STATUS_DISTRAINT_ACT_STATE;

select * from
gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_STATUS_DISTRAINT_ACT_STATE_ADS_MAP_SCD_DIFF
where tech_rid is not null;

---------------------------------------------------------------------------------------------
select xxhash64(uuid() ) 

SELECT 
*
FROM system.compute.warehouses 
WHERE warehouse_id = current_warehouse();

select new_customer_guid;
select current_warehouse();

SELECT as current_user;
---------------------------------------------------------------------------------------------
-- Disable Adaptive Query Execution (AQE) for small data
SET spark.sql.adaptive.enabled = false;
SET spark.sql.adaptive.coalescePartitions.enabled = false;
SET spark.sql.adaptive.localShuffleReader.enabled = false;
SET spark.sql.adaptive.skewJoin.enabled = false;

-- Reduce shuffle partitions for small data
SET spark.sql.shuffle.partitions = 8;  -- Default is 200, way too much for small tables

-- Disable cost-based optimizer for small queries
SET spark.sql.cbo.enabled = false;

-- Disable unnecessary optimizations for small writes
SET spark.databricks.delta.optimizeWrite.enabled = false;
SET spark.databricks.delta.autoCompact.enabled = false;

-- Speed up small file operations
SET spark.sql.files.maxPartitionBytes = 134217728;  -- 128MB instead of default 1GB
SET spark.sql.files.openCostInBytes = 4194304;      -- 4MB

-- Reduce broadcast join threshold for small tables
SET spark.sql.autoBroadcastJoinThreshold = 10485760; -- 10MB

-- Disable statistics collection for small tables
SET spark.sql.statistics.histogram.enabled = false;
SET spark.sql.statistics.fallBackToHdfs = false;

-------------------------------------
SET spark.sql.shuffle.partitions = 8;  -- This often works even in serverless
SET spark.sql.files.maxPartitionBytes = 67108864;
SET spark.databricks.optimizer.dynamicFilePruning = false;

--------------------------------------------------------------
-- Set properties on your target tables
ALTER TABLE gap_catalog.ads_owner.EVENT_TYPES SET TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'false',
    'delta.autoOptimize.autoCompact' = 'false',
    'delta.tuneFileSizesForRewrites' = 'false'
);

---------------------------------------------------------------
-- Use query hints instead of session settings
SELECT /*+ REPARTITION(1) */ * FROM small_table;

-- For joins, use broadcast hints

SELECT /*+ BROADCAST(small_table) */ 
    a.*, b.* 
FROM large_table a 
JOIN small_table b ON a.id = b.id;

-- Disable AQE at query level using hints
SELECT /*+ COALESCE(1) */ * FROM small_table;

------------------------------------------------------------
EXPLAIN EXTENDED 
--select /*+ COALESCE(1) */ * from gap_catalog.ads_owner.case_object_status;
insert  into gap_catalog.ads_owner.EVENT_TYPES 
 ( EVETP_KEY, 
   EVETP_SOURCE_ID, 
   EVETP_SOURCE_SYSTEM_ID, 
   EVETP_SOURCE_SYS_ORIGIN, 
   EVETP_DESC, 
   EVETP_EVENT_TABLE_NAME, 
   EVETP_VALID_FROM, 
   EVETP_VALID_TO, 
   EVETP_CURRENT_FLAG, 
   EVETP_DELETED_FLAG, 
   EVETP_INSERTED_DATETIME, 
   EVETP_INSERT_PROCESS_KEY, 
   EVETP_UPDATED_DATETIME, 
   EVETP_UPDATE_PROCESS_KEY, 
   EVETP_TARGET, 
   EVETP_DATA_PATH)
select /* + COALESCE(1) */ EVETP_KEY_NEW  AS EVETP_KEY,  -- Add the offset to maintain key sequence, 
   EVETP_SOURCE_ID, 
   EVETP_SOURCE_SYSTEM_ID, 
   EVETP_SOURCE_SYS_ORIGIN, 
   EVETP_DESC, 
   EVETP_EVENT_TABLE_NAME, 
   to_date(CAST('2025-08-14' AS DATE),'yyyy-MM-dd') as EVETP_VALID_FROM, 
   DATE('3000-01-01') as EVETP_VALID_TO, 
   'Y' as EVETP_CURRENT_FLAG, 
   tech_del_flg as EVETP_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as EVETP_INSERTED_DATETIME, 
   CAST(-999 AS BIGINT) as EVETP_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as EVETP_UPDATED_DATETIME, 
   CAST(-999 AS BIGINT) as EVETP_UPDATE_PROCESS_KEY, 
   EVETP_TARGET, 
   EVETP_DATA_PATH
  from gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_REGISTRATIONACTION_ADS_MAP_SCD_DIFF
 where tech_new_rec = 'Y';

 --------------------------------------------------------------------------------------
 -- Tag the session
SET spark.sql.execution.id = 'daily_etl_job_001';
SET custom.job.name = 'customer_data_refresh';
SET custom.execution.timestamp = current_timestamp();

-- Your queries here
DECLARE execution_id STRING DEFAULT 'ADS_RDS1';

SELECT /*'${execution_id}'*/
* FROM 
gap_catalog.ads_owner.case_status;

-- Query the system tables (available in Unity Catalog)
SELECT 
    statement_id,
    statement_text,
    start_time,
    end_time,
    TIMESTAMPDIFF(SECOND, start_time, end_time) as runtime_seconds,
    total_task_duration_ms,
    execution_status,
    compute,
executed_by,
    executed_by_user_id,
    statement_text,
    statement_type,
    total_duration_ms,
waiting_for_compute_duration_ms,
waiting_at_capacity_duration_ms,
    execution_duration_ms,
    compilation_duration_ms,
    total_task_duration_ms,
    result_fetch_duration_ms
    read_partitions,
    produced_rows,
    from_result_cache,
    CAST(statement_text AS varchar(100))
FROM system.query.history 
WHERE start_time >= current_date() - INTERVAL 1 DAY
   --AND statement_text LIKE '%ADS_RDS1%'
--ORDER BY start_time DESC
--AND statement_text <> ''
;
